In [17]:
import pickle as pkl
import pandas as pd
import re

In [18]:
with open('kanker_nl_predictions_no_dup.pkl', 'rb') as f:
    predictions = pkl.load(f)

In [19]:
len(predictions)

155691

In [20]:
predictions

[['<s> Voor sp ellen Ġtumoren Ġbij Ġvan ĠV on ĠH ip pel ĠL ind au Ġ </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>',
  ['O',
   'B',
   'B',
   'B',
   'B',
   'O',
   'O',
   'B',
   'B',
   'I',
   'I',
   'I',
   'I',
   'I',
   'I',
   'O',
   'O',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   'B',
   

In [8]:
for sentence, labels in predictions:
    tokens = sentence.split()
    
    zipped = zip(tokens,labels)
    for term in zipped:
        print(term)
    
    raise ValueError()
#     end_string = sentence.find('</s>')
#     sentence = sentence[:end_string]
    
#     end_labels = sentence.count(' ')
#     labels = labels[:end_labels]

#     sentence = sentence.replace(' ', '').replace('Ġ', ' ')

#     zipped = zip(sentence.split(),labels)
#     formatted = []
#     for token, label in zipped:
#         formatted.append(f"{token}({label})")
#     print(' '.join(formatted))
        

('<s>', 'O')
('Voor', 'B')
('sp', 'B')
('ellen', 'B')
('Ġtumoren', 'B')
('Ġbij', 'O')
('Ġvan', 'O')
('ĠV', 'B')
('on', 'B')
('ĠH', 'I')
('ip', 'I')
('pel', 'I')
('ĠL', 'I')
('ind', 'I')
('au', 'I')
('Ġ', 'O')
('</s>', 'O')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B')
('<pad>', 'B

ValueError: 

In [23]:
def convertLabel(label):
    if 'I' in label:
        return 'I'
    
    if 'B' in label:
        return 'B'
    
    return 'O'

In [24]:
def remove_punctuation(input_string):
    # Define regular expression pattern to match punctuation except dashes
    pattern = r'[^\w\s-]'
    
    # Use re.sub() function to replace matches with an empty string
    result = re.sub(pattern, '', input_string)
    
    return result

In [25]:
result = []

for sentence, labels in predictions:
    tokens = sentence.split()
    
    zipped = zip(tokens,labels)
    
    cleaned = []
    combined_term = ""
    combined_label = ""
    
    for term, label in reversed(list(zipped)[1:]):
        if term == '<pad>' or term == '</s>':
            continue
        
        combined_term = f"{term}{combined_term}"
        combined_label = f"{label}{combined_label}"
        
        # add colon
        if '.' == term:
            cleaned.append((combined_term.replace('Ġ', ''), combined_label))
            combined_term = ""
            combined_label = ""
        
        # if space, this is the combined token
        if 'Ġ' in term:
            cleaned.append((combined_term.replace('Ġ', ''), combined_label))
            combined_term = ""
            combined_label = ""
        
    
    cleaned = list(reversed(cleaned))
    formatted = []
    formatted_terms = []

    for i, (term, label) in enumerate(cleaned):
#       if first letter of sentence or words after O, and B or I -> B
        if (i == 0 and ('I' in label or 'B' in label)) or (i > 0 and formatted[-1][1] == 'O' and ('I' in label or 'B' in label)):
            new_label = 'B'
        elif 'I' in label and i > 0 and formatted[-1][1] == 'B':
            if label.count('B') > label.count('I'):
                new_label = 'B'
            else:
                new_label = 'I'
        elif 'B' in label:
            new_label = 'B'
        elif 'I' in label:
            new_label = 'I'
        else:
            new_label = 'O'
        
        formatted.append((term, new_label, label))
        formatted_terms.append(term)
#     for term, label, original_label in formatted:
#         print(term, label, original_label)
    
    mention = ""
    
#     print(formatted)
#     formatted_terms, _, _ = list(zip(*formatted))
    formatted_sentence = ' '.join(formatted_terms)
    
    for i, (term, label, _) in enumerate(formatted):
#         print(i, term, formatted, len(formatted))
        if i == len(formatted) - 1:
            if label == 'B':
                result.append([formatted_sentence, term]) 
            elif label == 'I':
                mention += f" {term}"
                result.append([formatted_sentence, mention.strip()]) 
            continue
        
        if label == 'B' and formatted[i+1][1] != 'I':
            result.append([formatted_sentence, term]) 
        elif label == 'B':
            mention = f"{term} "
        elif label == 'I':
            mention += f" {term}"
        
        if label == 'I' and formatted[i+1][1] != 'I':
            result.append([formatted_sentence, mention.strip()])
            mention = ''
            


results = pd.DataFrame(result)
results.columns = ['sentence', 'mention']
results['mention'] = results['mention'].str.replace('  ', ' ')
results['mention'] = results['mention'].str.replace('Ã«','ë')
results['mention'] = results['mention'].str.replace('Ã','ï')
results['mention'] = results['mention'].str.replace('Ċ','')
results['mention'] = results['mention'].str.replace('&nbsp;', ' ')

results['mention'] = results['mention'].apply(remove_punctuation)



In [26]:
results = results.drop_duplicates(subset=['sentence', 'mention'])
results

,sentence,mention
0,tumoren bij van Von Hippel Lindau,tumoren
1,tumoren bij van Von Hippel Lindau,Von Hippel Lindau
2,syndroom van Von Hippel Lindau (VHL) is zeldza...,syndroom van Von Hippel Lindau
3,syndroom van Von Hippel Lindau (VHL) is zeldza...,VHL
4,syndroom van Von Hippel Lindau (VHL) is zeldza...,zeldzaam
...,...,...
378182,"Fleckie,Wat goed dat jij in elk geval van meet...",triple negative tumor
378183,dat jouw therapie goed aanslaat!,therapie
378184,ben heel erg benieuwd wat er maandag ( overmor...,maandag
378185,ben heel erg benieuwd wat er maandag ( overmor...,uitslag


In [27]:
results.to_csv('kanker_entities_no_dup_sentences.csv', sep=';', escapechar='\\')

In [88]:
results

,sentence,mention
0,tumoren bij van Von Hippel Lindau,tumoren
1,tumoren bij van Von Hippel Lindau,Von Hippel Lindau
2,syndroom van Von Hippel Lindau (VHL) is zeldza...,syndroom van Von Hippel Lindau
3,syndroom van Von Hippel Lindau (VHL) is zeldza...,VHL
4,syndroom van Von Hippel Lindau (VHL) is zeldza...,zeldzaam
...,...,...
411445,"Fleckie,Wat goed dat jij in elk geval van meet...",triple negative tumor
411446,dat jouw therapie goed aanslaat!,therapie
411447,ben heel erg benieuwd wat er maandag ( overmor...,maandag
411448,ben heel erg benieuwd wat er maandag ( overmor...,uitslag


In [28]:
with open('kanker_entities_no_dup_sentences.pkl', 'wb') as f:
    pkl.dump(results, f)